In [ ]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import catboost
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

import optuna
from optuna.visualization import (
    plot_contour
    , plot_edf
    , plot_intermediate_values
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric

np.random.seed(2112)
pd.set_option('display.max_columns', None)

***
## load and prepare data

In [ ]:
!ls ../data/processed/dsv04

In [ ]:
train = pd.read_parquet("../data/processed/dsv04/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [ ]:
input_feats = train.columns.tolist()
len(input_feats)

In [ ]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
train = train.reset_index()

del train_labels
gc.collect()

***
## model tuning


In [ ]:
skf = StratifiedKFold(n_splits=3, random_state=2112, shuffle=True)
skf_split = list(skf.split(train, train["target"].values))

In [ ]:
default_params = {
    'learning_rate': 0.05,
    'nan_mode':'Min',
    'random_seed': 2112,
}

In [ ]:
def train_models(dataframe: pd.DataFrame, split: list, model_params: dict) -> pd.DataFrame:
        
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for train_idx,valid_idx in split:
        
        train_df = dataframe.loc[train_idx,:]
        valid_df = dataframe.loc[valid_idx,:]
                
        train_dset = catboost.Pool(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = catboost.Pool(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        
        model = catboost.CatBoostClassifier(**model_params)
        model.fit(train_dset, verbose=False)
                
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset, prediction_type="Probability")[:,1]

        del train_dset,valid_dset,model
        gc.collect()
    
    return oof

In [ ]:
def objective(trial):
    sampled_params = dict(
        # general booster config
        iterations = trial.suggest_int("iterations", 1000, 6000, 100),
        depth = trial.suggest_int("depth", 4, 8),
        rsm = trial.suggest_discrete_uniform("feature_fraction", 0.1, 0.5, 0.05),
        # regularization
        l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 0., 50.),
        min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 500, 3000, 100),
        bootstrap_type = trial.suggest_categorical("bootstrap_type", ["Bernoulli", "MVS"]),
        subsample = trial.suggest_discrete_uniform("subsample", 0.8, 1.0, 0.05),
        random_strength = trial.suggest_float("random_strength", 0., 50.),
        # class imbalance
        auto_class_weights = trial.suggest_categorical("auto_class_weights", [None, "Balanced", "SqrtBalanced"]),
    )
    model_params = {**default_params, **sampled_params}
    
    oof = train_models(train, skf_split, model_params)
    metric = compute_amex_metric(oof.target.values, oof.pred.values)
    return metric

In [ ]:
do_optimize = True

study = optuna.create_study(
    study_name="catboost-dsv04",
    direction='maximize',
    storage='sqlite:///catboost-dsv04.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=1000, 
        timeout=43200, # 12-hours
        n_jobs=1, 
        gc_after_trial=True,
    ) 

In [ ]:
study.trials_dataframe().sort_values("value", ascending=False).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_edf(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
best_params = dict(study.best_params)
best_params

***